In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import random
from selenium.webdriver.chrome.options import Options 
from pynput.keyboard import Key, Controller

In [2]:
# driver = webdriver.Chrome() 

In [3]:
driver = webdriver.Firefox()

In [4]:
driver.get('https://www.linkedin.com/login')

time.sleep(5)

username = driver.find_element("xpath","//input[@name='session_key']")
password = driver.find_element("xpath","//input[@name='session_password']")

time.sleep(5)



username.send_keys("alan@meditouch.co.il")

time.sleep(4)

password.send_keys('Alan_770770770')

time.sleep(2)

driver.find_element("xpath","//button[@type='submit']").click()
time.sleep(3)


In [5]:
# Specify the number of pages to navigate and the click limit
pages = 10
click_limit = 1
clicked_count = 0
clicked_names = set()

# Open the file in append mode to keep track of clicked names
with open('clicked_names.txt', 'a+') as file:
    # Load previously clicked names into a set to avoid duplicates
    file.seek(0)
    for line in file:
        clicked_names.add(line.strip())
    
    for n in range(1, pages + 1):
        if n == 1:
            driver.get("https://www.linkedin.com/sales/lists/people/7152301656594919424?sortCriteria=CREATED_TIME&sortOrder=DESCENDING")
        else:
            driver.get(f"https://www.linkedin.com/sales/lists/people/7152301656594919424?page={n}&sortCriteria=CREATED_TIME&sortOrder=DESCENDING")
        time.sleep(10)
        
        # Locate all dropdown buttons
        buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'artdeco-dropdown__trigger'))
        )
        
        # Locate all degree spans
        degrees = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//li[starts-with(@class,'list-lead-detail__degree-badge')]//span[@class='a11y-text']"))
        )
        
        # Locate all names
        names = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//a[@data-anonymize='person-name']"))
        )
        
        # Check if the number of spans and buttons are equal
        if len(buttons) - 1 != len(degrees) or len(degrees) != len(names):
            print("Mismatch between buttons, spans, and names. Moving to the next page.")
            continue
        
        # Loop through buttons starting from index 1
        for i in range(1, len(buttons)):
            try:
                name = names[i - 1].text
                if name in clicked_names:
                    print(f"Skipping {name}, already clicked.")
                    continue

                first_name = name.split(" ")[0]
                # Check if the corresponding span has "1st degree contact"
                if degrees[i - 1].text == "1st degree contact":  # i-1 because spans are related to buttons[i] starting from index 1
                    # Click the dropdown button to open the menu
                    trigger_button = buttons[i]
                    trigger_button.click()
        
                    # Wait for dropdown items to be visible and locate the first item ("Message")
                    items = WebDriverWait(driver, 3).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'artdeco-dropdown__item'))
                    )
                    items[0].click()  # Click the first item (Message)
                    for item in items:
                        print(f"{items.index(item)}, {item.text}")
                    time.sleep(5)  # Wait for the message form to open
        
                    # Locate and click the message textarea (or other field as required)
                    message_area = driver.find_element(By.TAG_NAME, 'textarea')
                    message_area.click()
        
                    # Optional: Type a message into the textarea
                    # message_area.send_keys(f"Hi, {first_name}.\nAfter assessment, it is key to train balance and gait.\nUsing multi directional gait perturbations with the balanceTutor is a fun and effective way to improve balance and gait.\nhttps://bit.ly/423Ngx9")
                    file_input = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.ID, "attachment"))
                    )
                    file_input.send_keys("/home/romeo/Pictures/Screenshot_20240901_093448.png")
                    time.sleep(5)

                    time.sleep(5)
                    # Optional: Click the send button if you want to send the message
                    send_button = driver.find_element(By.XPATH, "//button[starts-with(@class, 'ember-view _button_ps32ck _small_ps32ck _primary_ps32ck _left_ps32ck _container_iq15dg ml3')]")
                    time.sleep(3)
                    # if send_button:
                    #     send_button.click()
        
                    time.sleep(10)  # Wait for the message to send (if applicable)
        
                    # Close the message form
                    close_button = WebDriverWait(driver, 7).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[starts-with(@data-control-name, 'overlay.close_overlay')]"))
                    )
                    close_button.click()
        
                    time.sleep(5)  # Wait for the form to close before moving to the next dropdown button
                    
                    # Record the clicked name and increment the count
                    file.write(name + '\n')
                    clicked_names.add(name)
                    clicked_count += 1
                    
                    # Check if the click limit has been reached
                    if clicked_count >= click_limit:
                        print(f"Click limit of {click_limit} reached. Exiting.")
                        break
        
                else:
                    print(f"Skipping button index {i}, span text is not '1st degree contact'")
        
            except Exception as e:
                print(f"An error occurred with button index {i}: {str(e)}")
                continue  # Continue to the next button even if an error occurs
        
        # Exit if the click limit is reached
        if clicked_count >= click_limit:
            break

An error occurred with button index 1: Message: The element with the reference d345c441-a9b7-41c9-9649-a2ac9a641f27 is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:659:5
element.getKnownElement@chrome://remote/content/marionette/element.sys.mjs:509:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:233:33
cloneObject@chrome://remote/content/marionette/json.sys.mjs:56:24
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:244:16
json.deserialize@chrome://remote/content/marionette/json.sys.mjs:248:10
receiveMessage@chrom

KeyboardInterrupt: 

In [ ]:
# driver.quit()